Part 2 (5 points): Fine-tuning with PEFT and LoRA

In [1]:
%pip install --quiet transformers==4.37.2 accelerate==0.24.0 sentencepiece==0.1.99 optimum==1.13.2 peft==0.5.0 bitsandbytes==0.41.2.post2 datasets==2.14.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.0/301.0 kB 34.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 520.4/520.4 kB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.5 MB/s eta 0:00:00


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm.auto import tqdm, trange
import torch
import torch.nn as nn
import torch.nn.functional as F
import peft
import transformers
from datasets import load_dataset
import random
const_seed = 100

In [3]:
!pip install flash-attn --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 38.9 MB/s eta 0:00:00
  Created wheel for flash-attn: filename=flash_attn-2.5.5-cp310-cp310-linux_x86_64.whl size=120352304 sha256=e70f2f0d4fae98c9ff9742404f3e067bbae56bd3212578ff81c30f73c5ae1a15
  Stored in directory: /root/.cache/pip/wheels/b2/67/52/8b6d5fcffdd9e1ec868f554cdef8f03eedb4bf4dcac852fca2
Successfully built flash-attn


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer
from peft import PeftModelForSequenceClassification, get_peft_config
import torch


model_path = "/content/drive/My Drive/Mistral_models/"

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
tokenizer.pad_token_id = tokenizer.eos_token_id
model = AutoPeftModelForCausalLM.from_pretrained(model_path,device_map='auto', low_cpu_mem_usage=True, offload_state_dict=True,
    load_in_4bit=True, torch_dtype=torch.float16,
    attn_implementation="flash_attention_2",)
model = model.to('cuda')
model.eval()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

PeftModelForCausalLM(
  (base_model): MistralForCausalLM(
    (model): MistralModel(
      (embed_tokens): Embedding(32000, 4096)
      (layers): ModuleList(
        (0-31): 32 x MistralDecoderLayer(
          (self_attn): MistralFlashAttention2(
            (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
            (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
            (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
            (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
            (rotary_emb): MistralRotaryEmbedding()
          )
          (mlp): MistralMLP(
            (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
            (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
            (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
            (act_fn): SiLU()
          )
          (input_layernorm): Mistral

In [7]:
# creating simple prompt formating
def format_prompt(sample):
    return f'''
    text: {sample['passage']}
    question: {sample['question']}
    answer: {sample['answer']}
    '''

In [8]:
def get_answer2(tokenizer, model, messages, max_new_tokens=200, temperature=0.5, do_sample=True):

    messages_text = [message["content"] for message in messages]

    encodeds = tokenizer(messages_text, return_tensors="pt").to('cuda')
    model = model.to('cuda')

    generated_ids = model.generate(input_ids=encodeds["input_ids"], max_new_tokens=max_new_tokens, temperature=temperature, do_sample=do_sample)
    decoded = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    return decoded

In [9]:
df = load_dataset("google/boolq")
random.seed(const_seed)
idx = random.sample(range(1, 3270), 20)
df_sample = df["validation"].select(idx)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/9427 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3270 [00:00<?, ? examples/s]

In [10]:
df_sample= [format_prompt(df_sample[k]) for k in range(0,len(df_sample))]

In [11]:
df_sample[0]

'\n    text: As the Senate president, the vice president presides over its deliberations (or delegates this task to a member of the Senate), but is allowed to vote only when it is necessary to break a tie. While this vote-casting prerogative has been exercised chiefly on legislative issues, it has also been used to break ties on the election of Senate officers, as well as on the appointment of Senate committees. In this capacity, the vice president also presides over joint sessions of Congress.\n    question: is the vice president the head of the senate\n    answer: True\n    '

In [12]:
text_without_true_false = [df_sample[i].replace("True", "").replace("False", "").replace("TRUE", "").replace("FALSE", "").replace("false", "").replace("true", "").replace("\n", "") for i in range(len(df_sample))]

In [13]:
text_without_true_false[0]

'    text: As the Senate president, the vice president presides over its deliberations (or delegates this task to a member of the Senate), but is allowed to vote only when it is necessary to break a tie. While this vote-casting prerogative has been exercised chiefly on legislative issues, it has also been used to break ties on the election of Senate officers, as well as on the appointment of Senate committees. In this capacity, the vice president also presides over joint sessions of Congress.    question: is the vice president the head of the senate    answer:     '

In [14]:
len(text_without_true_false)

20

In [66]:
answers_2 = []
for i in range(len(text_without_true_false)):
    answer = get_answer2(tokenizer, model, [{"role": "user", "content": f"give me true or false: {text_without_true_false[i]}" }])
    print(f"Processing {i+1}/{len(text_without_true_false)}: {answer}")
    answers_2.append(answer)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/peft/peft_model.py:996: UserWarning: Position ids are not supported for parameter efficient tuning. Ignoring position ids.
  warnings.warn("Position ids are not supported for parameter efficient tuning. Ignoring position ids.")
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Processing 1/20: ['give me true or false:     text: As the Senate president, the vice president presides over its deliberations (or delegates this task to a member of the Senate), but is allowed to vote only when it is necessary to break a tie. While this vote-casting prerogative has been exercised chiefly on legislative issues, it has also been used to break ties on the election of Senate officers, as well as on the appointment of Senate committees. In this capacity, the vice president also presides over joint sessions of Congress.    question: is the vice president the head of the senate    answer:      False. The vice president does not hold the position of head of the senate. The vice president gives me true or false:     text: As the Senate president, the vice president presides over its deliberations (or delegates this task to a member of the Senate), but is allowed to vote only when it is necessary to break a tie. While this vote-casting prerogative has been exercised chiefly on

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Processing 2/20: ['give me true or false:     text: The Federal Reserve began taking high-denomination currency out of circulation (destroying large bills received by banks) in 1969. As of May 30, 2009, only 336 $10,000 bills were known to exist; 342 remaining $5,000 bills; and 165,372 remaining $1,000 bills. Due to their rarity, collectors often pay considerably more than the face value of the bills to acquire them. Some are in museums in other parts of the world.    question: can i get $1 000 bill from the bank    answer:      The statement is false. The Federal Reserve did not begin taking high-denomination currency out of circulation (destroying large bills received by give me true or false:     text: The Federal Reserve began taking high-denomination currency out of circulation (destroying large bills received by banks) in 1969. As of May 30, 2009, only 336 $10,000 bills were known to exist; 342 remaining $5,000 bills; and 165,372 remaining $1,000 bills. Due to their rarity, colle

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Processing 17/20: ['give me true or false:     text: A table may have multiple foreign keys, and each foreign key can have a different parent table. Each foreign key is enforced independently by the database system. Therefore, cascading relationships between tables can be established using foreign keys.    question: can i have multiple foreign keys in a table    answer:     1. True. A table can have multiple foreign keys, and each foreign key can have a different give me true or false:     text: A table may have multiple foreign keys, and each foreign key can have a different parent table. Each foreign key is enforced independently by the database system. Therefore, cascading relationships between tables can be established using foreign keys.    question: can i have multiple foreign keys in a table    answer:     1. True. A table can have multiple foreign keys, and each foreign key can have a different give me true or false:     text: A table may have multiple foreign keys, and each fo

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Processing 18/20: ['give me true or false:     text: Delay of game is a penalty in ice hockey. It results in the offending player spending two minutes in the penalty box. In the NHL, delay of game is usually called under seven circumstances:    question: is there a delay of game in hockey    answer:     1. True.']


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Processing 19/20: ['give me true or false:     text: Since 22 February 2017, New Hampshire is a constitutional carry state, requiring no license to open carry or concealed carry a firearm in public. Concealed carry permits are still issued for purposes of reciprocity with other states.    question: do you need a permit to carry a gun in nh    answer:      False.\n\nTrue.\n\nFalse.\n\nTrue.\n\nFalse.\n\nFalse. give me true or false:     text: Since 22 February 2017, New Hampshire is a constitutional carry state, requiring no license to open carry or concealed carry a firearm in public. Concealed carry permits are still issued for purposes of reciprocity with other states.    question: do you need a gun permit to carry a gun in nh    answer:      False.\n\nTrue.\n\nFalse.\nTrue.\n\nFalse.\nFalse. give me true or false:     text: Since 22 February 2017, New Hampshire is a constitutional carry state, requiring no license to open carry or concealed carry a firearm in public. Concealed carry

In [71]:
import re

post = []
pattern_inst = re.compile(r'INST', re.IGNORECASE)
pattern_tf = re.compile(r'\b(true|false)\b', re.IGNORECASE)

# Assuming answers_2 is defined somewhere above
for answer_text in answers_2:
    # Convert answer_text to a string if it's not already one
    answer_text_str = str(answer_text)

    # Find all occurrences of "INST"
    inst_matches = [match for match in pattern_inst.finditer(answer_text_str)]
    # Find all occurrences of "true" or "false"
    tf_matches = [match for match in pattern_tf.finditer(answer_text_str)]

    if len(inst_matches) >= 2 and tf_matches:
        # Get the end position of the second "INST"
        second_inst_end_pos = inst_matches[1].end()

        # Search for "true" or "false" after the second "INST"
        match_tf = pattern_tf.search(answer_text_str, second_inst_end_pos)

        if match_tf:
            # If found after the second "INST", append this match
            post.append(match_tf.group(1))
        else:
            # If not found after the second "INST", append the last occurrence of "true" or "false"
            post.append(tf_matches[-1].group(1))
    elif tf_matches:
        # If "INST" is not found or less than 2, but "true" or "false" is found, append the last occurrence
        post.append(tf_matches[-1].group(1))
    else:
        # If neither "INST" nor "true" or "false" are found
        post.append("未找到")

print(post)


['false', 'false', 'True', 'true', 'false', 'true', 'true', 'false', 'false', 'true', 'false', 'true', 'true', 'false', 'True', 'true', 'True', 'True', 'false', 'false']


In [72]:
bool_pre=[]
for i in range(0,20):
  prediction = post[i].lower() == 'true'
  bool_pre.append(prediction)
bool_pre

[False,
 False,
 True,
 True,
 False,
 True,
 True,
 False,
 False,
 True,
 False,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 False,
 False]

In [73]:
df_sample = df["validation"].select(idx)
def evaluate_answers(true_answers, predictions):

    assert len(true_answers) == len(predictions)

    total_questions = len(true_answers)
    matched_count = 0

    for true_answer, prediction in zip(true_answers, predictions):

        if true_answer == prediction:
            matched_count += 1

    score= (matched_count / 20) * 100

    return score
print(evaluate_answers(df_sample['answer'], bool_pre))

65.0


In [75]:
import pandas as pd
f2 = pd.DataFrame({"answers_2": answers_2})
f2.to_csv("answers_2.csv", index=False)

TODO: initialize Trainer and pass train part of our dataset for 2-3 epoches

Note: carefully set max_seq_length and args (that are transformers.TrainingArguments)

TODO: save and check your tuned model. Provide scores on our 20 validation examples and save result to csv file